## Definición de los estimadores de pose

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil

# Función para dibujar puntos de la pose del cuerpo
def draw_pose_landmarks(frame, landmarks, COLOR):
    height, width, _ = frame.shape
    for point in landmarks.landmark:
        x, y = int(point.x * width), int(point.y * height)
        cv2.circle(frame, (x, y), 5, COLOR, -1)

# Función para dibujar puntos y líneas de las manos
def draw_hand_landmarks_with_square(frame, landmarks, COLOR):
    height, width, _ = frame.shape
    x_min, y_min, x_max, y_max = width, height, 0, 0
    
    # Dibujar puntos y líneas de las manos
    for i, point in enumerate(landmarks.landmark):
        x, y = int(point.x * width), int(point.y * height)
        color = (0, 255, 0) if i % 4 == 0 else (0, 0, 255)
        cv2.circle(frame, (x, y), int(height * 0.01), color, -1)
        # Actualizar los límites del cuadrado
        x_min = min(x_min, x)
        y_min = min(y_min, y)
        x_max = max(x_max, x)
        y_max = max(y_max, y)
    
    connections = [[0, 1], [1, 2], [2, 3], [3, 4], [0, 5], [5, 6], [6, 7], [7, 8], [0, 9], [9, 10], [10, 11], [11, 12], [0, 13], [13, 14], [14, 15], [15, 16], [0, 17], [17, 18], [18, 19], [19, 20]]
    for connection in connections:
        cv2.line(frame, (int(landmarks.landmark[connection[0]].x * width), int(landmarks.landmark[connection[0]].y * height)),
                 (int(landmarks.landmark[connection[1]].x * width), int(landmarks.landmark[connection[1]].y * height)), COLOR, 2)
    cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (255, 0, 255), 2)

# Función para dibujar cuadro de la cara
def draw_face_detection(frame, detections, COLOR):
    height, width, _ = frame.shape
    for detection in detections:
        bboxC = detection.location_data.relative_bounding_box
        ih, iw, _ = frame.shape
        bbox = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
        cv2.rectangle(frame, bbox, COLOR, 2)

# Inicializar los modelos de MediaPipe
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_face = mp.solutions.face_detection

pose = mp_pose.Pose()
hands = mp_hands.Hands()
face_detection = mp_face.FaceDetection()

# Constantes para colores
RED = (0, 0, 255)
GREEN = (0, 255, 0)
BLUE = (255, 0, 0)


## Captura en tiempo real de la pose de una seña

In [3]:

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error al abrir la cámara")
else:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error al leer el frame")
            break

        # height, width, _ = frame.shape

        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detección de la pose del cuerpo
        pose_results = pose.process(img)

        # Detección de las manos
        hands_results = hands.process(img)

        # Detección de la cara
        face_results = face_detection.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Crear una imagen negra del mismo tamaño que el frame
        black_frame = np.zeros_like(frame)

        # Dibujar puntos y líneas para la pose del cuerpo en la imagen negra
        # if pose_results.pose_landmarks:
        #     draw_pose_landmarks(black_frame, pose_results.pose_landmarks)

        # Dibujar círculos y líneas para las manos en la imagen negra
        if hands_results.multi_hand_landmarks:
            for landmarks in hands_results.multi_hand_landmarks:
                draw_hand_landmarks(black_frame, landmarks, BLUE)

        # Dibujar puntos y cuadro de la cara en la imagen original
        # if face_results.detections:
        #     draw_face_detection(frame, face_results.detections)

        cv2.imshow('Pose Detection', black_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

NameError: name 'draw_hand_landmarks' is not defined

: 

## Uso del modelo en tiempo real

In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2


model = load_model('model.h5')

class_indices = json.load(open('class_indices.json'))

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    img = cv2.resize(frame, (224, 224))
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    prediction = model.predict(img)
    predicted_class = list(class_indices.keys())[np.argmax(prediction)]
    cv2.putText(frame, predicted_class, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

model = load_model('model.h5')

class_indices = json.load(open('class_indices.json'))

def predict_image(model, img_path, class_indices):
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    prediction = model.predict(img)
    predicted_class = list(class_indices.keys())[np.argmax(prediction)]
    return predicted_class

img_path = "./classes_original/M/DSC01254.JPG"
predicted_class = predict_image(model, img_path, class_indices)
print(predicted_class)
